# Import libaries and read the listing table

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
df = pd.read_csv('C:/Users/Nancy Zhao/Desktop/Nano degrees/Data science/CRISP-DM/data science blog post/listings.csv')
df = df[['id', 'host_response_rate', 'neighbourhood_cleansed', 'property_type', 'room_type', 'price', 'weekly_price', 'monthly_price', 'review_scores_rating', 'review_scores_location','cancellation_policy']]
df.head()

In [ ]:
# check dtypes for the variables and see whether a conversion is needed.
df.info()

# Data preparation for the listings table: Convert data to appropriate dtype

In [ ]:
df['host_response_rate_val'] = df['host_response_rate'].str.rstrip('%').astype(float)/100
df['price_val'] = df['price'].str.lstrip('$').str.replace(',', '').astype(float)
df['weekly_price_val'] = df['weekly_price'].str.lstrip('$').str.replace(',', '').astype(float)

In [ ]:
df.head()

# Data Analysis

## Question 1: Number of Airbnb properties by neighborhood

In [ ]:
#How many Airbnb properties in Seattle
total_properties = len(np.unique(df['id']))
total_properties

In [ ]:
#number of properties by neighborhood
number_properties = pd.DataFrame(df.groupby(df['neighbourhood_cleansed'])['id'].count().sort_values(ascending = False))
number_properties = number_properties.rename(columns = {'id': "Number of properties"}) 
number_properties

In [ ]:
# number of properties in a few communities
print(number_properties.loc[['Pike-Market', 'Alki', 'Fauntleroy', 'Highland Park', 'South Beacon Hill', 'South Park']])

In [ ]:
# Market share by neighborhood
proportion_properties = number_properties/number_properties.sum()
proportion_properties = proportion_properties.rename(columns = {'id': 'proportion'})
proportion_properties

In [ ]:
# Market share for a few communities
proportion_properties.loc[['Montlake', 'Westlake', 'South Lake Union', 'Madrona', 'East Queen Anne', 'North Beach/Blue Ridge', 'Belltown', 'West Queen Anne']]

In [ ]:
# Market share for the top three communities
proportion_properties.iloc[0:3].sum()

## Question 2: Ten top rated neighbourhood by location

In [ ]:
rating_by_neighbor = pd.DataFrame(df.groupby(df['neighbourhood_cleansed'])['review_scores_location'].mean().sort_values(ascending = False))
rating_by_neighbor

In [ ]:
#Concat rating and number of properties: how saturated the market is in the communities of 'good location'
rating_properties = pd.merge(rating_by_neighbor, number_properties, on = 'neighbourhood_cleansed')
rating_properties.iloc[0:11]

# Data preparation for the calendar table

In [ ]:
df1 = pd.read_csv('C:/Users/Nancy Zhao/Desktop/Nano degrees/Data science/CRISP-DM/data science blog post/calendar.csv')
df1.info()

## Truncate date by month

In [ ]:
df1['date_trans'] = pd.to_datetime(df1['date'])
df1['date_month'] = df1['date_trans'].apply(lambda x: x.strftime('%m'))
len(df1['date_trans']) # 1393570 entries in total

## Question 3  Occupancy rate and monthly revenue by neighbourhood in 2016

## Slice the data of 2016 and convert 'available' from "t and f" to "0 and 1"

In [ ]:
df1 = df1[(df1['date_trans'] >= pd.datetime(2016,1,1)) & (df1['date_trans'] <= pd.datetime(2016,12,31))]
df1['available_bool'] = np.where(df1['available'].str.contains('f'), 1,0)

## Merge the listing table and the calendar table by id

In [ ]:
df2 = pd.merge(df, df1, left_on = 'id', right_on = 'listing_id')
df2.head()

## Calculate the occupancy rate by neighbourhood

In [ ]:
occupancy_neighbour = df2.groupby(df2['neighbourhood_cleansed'])['available_bool'].sum()
total_neighbour = df2.groupby(df2['neighbourhood_cleansed'])['available_bool'].count()
occupancy_rate_neighbour = occupancy_neighbour / total_neighbour
occupancy_rate_neighbour.sort_values(ascending = False)

## Calculate the average price by neighbourhood

In [ ]:
price_by_neighbor = df2.groupby(df2['neighbourhood_cleansed'])['price_val'].mean()
price_by_neighbor.sort_values(ascending = False)

In [ ]:
#Get prices for a few communities
price_by_neighbor.loc[['Pike-Market', 'Alki', 'Fauntleroy']]

## Calculate monthly reveunue by neighbourhood

In [ ]:
monthly_income_neighbour = 30*occupancy_rate_neighbour*price_by_neighbor
monthly_income_neighbour = pd.DataFrame(monthly_income_neighbour.sort_values(ascending = False))
monthly_income_neighbour = monthly_income_neighbour.rename(columns = {0: 'monthly revenue'})
monthly_income_neighbour

## Bar plot the five most profitable Airbnb properties by neighborhood

In [ ]:
g_income = monthly_income_neighbour.head(n=5)
g_income.info()
g_income.plot.bar()

In [ ]:
#Get monthly revenue for a few communities
monthly_income_neighbour.loc[['Broadway', 'Belltown', 'Wallingford', 'Pike-Market', 'Alki', 'Fauntleroy']]

In [ ]:
#Merge revenue and proportion to see: how profitable the properties are by neighbourhood? How saturated the market is? (Any potential for market entry)
revenue_proportion = pd.merge(monthly_income_neighbour, proportion_properties, on = 'neighbourhood_cleansed')
revenue_proportion

## Question 4:  Monthly revenue by property type

## Occupancy rate by property

In [ ]:
occupancy_property = df2.groupby(df2['property_type'])['available_bool'].sum()
total_property = df2.groupby(df2['property_type'])['available_bool'].count()
occupancy_rate_property = occupancy_property / total_property
occupancy_rate_property1 = pd.DataFrame(occupancy_rate_property.sort_values(ascending = False))
occupancy_rate_property1 = occupancy_rate_property1.rename(columns = {'available_bool' : 'occupancy rate'})
occupancy_rate_property1

## Average price by property

In [ ]:
avg_price_property = df2.groupby(df2['property_type'])['price_val'].mean().sort_values(ascending = False)
avg_price_property1 = pd.DataFrame(avg_price_property )
avg_price_property1 = avg_price_property1.rename(columns = {'price_val' : 'price per night'})
avg_price_property1

## Monthly revenue by property

In [ ]:
monthly_income_property = 30*occupancy_rate_property*avg_price_property
monthly_income_property = pd.DataFrame(monthly_income_property.sort_values(ascending = False))
monthly_income_property = monthly_income_property.rename(columns = {0 : 'monthly revenue'})
monthly_income_property

In [ ]:
#merge monthly revenue, price, occupancy rate by property for better presentation
revenue_property = pd.merge(pd.merge(monthly_income_property,avg_price_property1,on = 'property_type'), occupancy_rate_property1, on = 'property_type')
revenue_property

## Bar plot monthly revenue by property type

In [ ]:
monthly_bar = sns.barplot(monthly_income_property.index, monthly_income_property.values)
monthly_bar.set_xticklabels(monthly_bar.get_xticklabels(), rotation = 90)